In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, when, lit, isnull, count, current_date, months_between, to_date
from pyspark.sql.types import FloatType

spark = SparkSession.builder.appName("EmployeeProjectAnalysis").getOrCreate()
employee_data = [
    ("Ananya", "HR", 52000),
    ("Rahul", "Engineering", 65000),
    ("Priya", "Engineering", 60000),
    ("Zoya", "Marketing", 48000),
    ("Karan", "HR", 53000),
    ("Naveen", "Engineering", 70000),
    ("Fatima", "Marketing", 45000)
]
columns_emp = ["Name", "Department", "Salary"]
df_emp = spark.createDataFrame(employee_data, columns_emp)
performance_data = [
    ("Ananya", 2023, 4.5),
    ("Rahul", 2023, 4.9),
    ("Priya", 2023, 4.3),
    ("Zoya", 2023, 3.8),
    ("Karan", 2023, 4.1),
    ("Naveen", 2023, 4.7),
    ("Fatima", 2023, 3.9)
]
columns_perf = ["Name", "Year", "Rating"]
df_perf = spark.createDataFrame(performance_data, columns_perf)
project_data = [
    ("Ananya", "HR Portal", 120),
    ("Rahul", "Data Platform", 200),
    ("Priya", "Data Platform", 180),
    ("Zoya", "Campaign Tracker", 100),
    ("Karan", "HR Portal", 130),
    ("Naveen", "ML Pipeline", 220),
    ("Fatima", "Campaign Tracker", 90)
]
columns_proj = ["Name", "Project", "HoursWorked"]
df_proj = spark.createDataFrame(project_data, columns_proj)


 1. Joins and Advanced Aggregations


In [0]:
df_merged = df_emp.join(df_perf, "Name").join(df_proj, "Name")
df_merged.show()


+------+-----------+------+----+------+----------------+-----------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|
+------+-----------+------+----+------+----------------+-----------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|
+------+-----------+------+----+------+----------------+-----------+



In [0]:
df_merged.groupBy("Department").sum("HoursWorked").withColumnRenamed("sum(HoursWorked)", "TotalHours").show()



+-----------+----------+
| Department|TotalHours|
+-----------+----------+
|         HR|       250|
|Engineering|       600|
|  Marketing|       190|
+-----------+----------+



In [0]:
df_merged.groupBy("Project").agg(avg("Rating").alias("AvgRating")).show()


+----------------+------------------+
|         Project|         AvgRating|
+----------------+------------------+
|       HR Portal|               4.3|
|   Data Platform|               4.6|
|Campaign Tracker|3.8499999999999996|
|     ML Pipeline|               4.7|
+----------------+------------------+



Handling Missing Data (introduce some manually)

In [0]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Rating", FloatType(), True),  
    FloatType
])

new_row = spark.createDataFrame([("Dummy", 2023, None)], schema=schema)
df_perf_null = df_perf.unionByName(new_row)
df_perf_null.show()


+------+----+------+
|  Name|Year|Rating|
+------+----+------+
|Ananya|2023|   4.5|
| Rahul|2023|   4.9|
| Priya|2023|   4.3|
|  Zoya|2023|   3.8|
| Karan|2023|   4.1|
|Naveen|2023|   4.7|
|Fatima|2023|   3.9|
| Dummy|2023|  NULL|
+------+----+------+



In [0]:
df_perf_null.filter(col("Rating").isNull()).show()


+-----+----+------+
| Name|Year|Rating|
+-----+----+------+
|Dummy|2023|  NULL|
+-----+----+------+



In [0]:

df_temp = df_perf_null.join(df_emp, "Name", "left")
dept_avg = df_temp.groupBy("Department").agg(avg("Rating").alias("DeptAvg"))
df_temp = df_temp.join(dept_avg, "Department", "left")
df_filled = df_temp.withColumn(
    "RatingFilled",
    when(col("Rating").isNull(), col("DeptAvg")).otherwise(col("Rating"))
).select("Name", "Department", "RatingFilled")

df_filled.show()


+------+-----------+------------+
|  Name| Department|RatingFilled|
+------+-----------+------------+
|Ananya|         HR|         4.5|
| Rahul|Engineering|         4.9|
| Priya|Engineering|         4.3|
|  Zoya|  Marketing|         3.8|
| Karan|         HR|         4.1|
|Naveen|Engineering|         4.7|
|Fatima|  Marketing|         3.9|
| Dummy|       NULL|        NULL|
+------+-----------+------------+



Built-In Functions and UDF

In [0]:
df_cat = df_perf.withColumn(
    "PerformanceCategory",
    when(col("Rating") >= 4.7, "Excellent")
    .when(col("Rating") >= 4.0, "Good")
    .otherwise("Average")
)
df_cat.show()


+------+----+------+-------------------+
|  Name|Year|Rating|PerformanceCategory|
+------+----+------+-------------------+
|Ananya|2023|   4.5|               Good|
| Rahul|2023|   4.9|          Excellent|
| Priya|2023|   4.3|               Good|
|  Zoya|2023|   3.8|            Average|
| Karan|2023|   4.1|               Good|
|Naveen|2023|   4.7|          Excellent|
|Fatima|2023|   3.9|            Average|
+------+----+------+-------------------+



In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def bonus_udf(hours):
    return 10000 if hours > 200 else 5000

bonus_func = udf(bonus_udf, IntegerType())
df_with_bonus = df_proj.withColumn("Bonus", bonus_func(col("HoursWorked")))
df_with_bonus.show()


+------+----------------+-----------+-----+
|  Name|         Project|HoursWorked|Bonus|
+------+----------------+-----------+-----+
|Ananya|       HR Portal|        120| 5000|
| Rahul|   Data Platform|        200| 5000|
| Priya|   Data Platform|        180| 5000|
|  Zoya|Campaign Tracker|        100| 5000|
| Karan|       HR Portal|        130| 5000|
|Naveen|     ML Pipeline|        220|10000|
|Fatima|Campaign Tracker|         90| 5000|
+------+----------------+-----------+-----+



Date and Time Functions

In [0]:
from pyspark.sql.functions import months_between

df_emp_date = df_emp.withColumn("JoinDate", to_date(lit("2021-06-01")))
df_emp_date = df_emp_date.withColumn("MonthsWorked", months_between(current_date(), col("JoinDate")).cast("int"))
df_emp_date.select("Name", "JoinDate", "MonthsWorked").show()


+------+----------+------------+
|  Name|  JoinDate|MonthsWorked|
+------+----------+------------+
|Ananya|2021-06-01|          48|
| Rahul|2021-06-01|          48|
| Priya|2021-06-01|          48|
|  Zoya|2021-06-01|          48|
| Karan|2021-06-01|          48|
|Naveen|2021-06-01|          48|
|Fatima|2021-06-01|          48|
+------+----------+------------+



In [0]:
df_emp_date.filter(col("JoinDate") < "2022-01-01").count()


7

Unions

In [0]:
extra_employees = [
    ("Meena", "HR", 48000),
    ("Raj", "Marketing", 51000)
]
df_extra = spark.createDataFrame(extra_employees, columns_emp)
df_union = df_emp.unionByName(df_extra)
df_union.show()



+------+-----------+------+
|  Name| Department|Salary|
+------+-----------+------+
|Ananya|         HR| 52000|
| Rahul|Engineering| 65000|
| Priya|Engineering| 60000|
|  Zoya|  Marketing| 48000|
| Karan|         HR| 53000|
|Naveen|Engineering| 70000|
|Fatima|  Marketing| 45000|
| Meena|         HR| 48000|
|   Raj|  Marketing| 51000|
+------+-----------+------+



Saving Results

In [0]:
df_final = df_emp.join(df_perf, "Name").join(df_proj, "Name")
df_final.write.mode("overwrite").parquet("/tmp/final_employee_dataset")
